In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('../clean_data/forced_choice_emotion_uw_students.csv')
df_labels = pd.read_csv('../data/emotion_labels.csv')

In [37]:
df_emo_answers = df.loc[:, 'Q2.1':'Q195.1'] # subset photos

# Long format DF

In [38]:
df_emo_answers_long = df_emo_answers.reset_index()
df_emo_answers_long['index'] = df_emo_answers_long.index
df_emo_answers_long = df_emo_answers_long.rename(columns={'index':'participantId'})
df_emo_answers_long.head(2)

,participantId,Q2.1,Q3.1,Q4.1,Q5.1,Q6.1,Q7.1,Q8.1,Q9.1,Q10.1,...,Q186.1,Q187.1,Q188.1,Q189.1,Q190.1,Q191.1,Q192.1,Q193.1,Q194.1,Q195.1
0,0,Neutral,Anger,Anger,Anger,Anger,Anger,Anger,Happiness,Disgust,...,Surprise,Neutral,Happiness,Happiness,Other,Other,Uncertain,Uncertain,Other,Neutral
1,1,Anger,Anger,Anger,Sadness,Anger,Anger,Fear,Anger,Other,...,Surprise,Surprise,Happiness,Happiness,Disgust,Uncertain,Other,Anger,Uncertain,Uncertain


In [39]:
df_labels['photoId'] =  df_labels['photoId'].str.strip()
photoIds = df_labels['photoId'].tolist()

In [40]:
df_emo_answers_long = pd.melt(df_emo_answers_long, id_vars=['participantId'], value_vars=photoIds).reset_index(drop=True).sort_values(by=['participantId'])

In [41]:
df_emo_answers_long = df_emo_answers_long.rename(columns={'variable':'photoId', 'value':'emotion'})

In [42]:
df_emo_answers_long_m = pd.merge(df_emo_answers_long, df_labels, how="left", on=["photoId"], validate="many_to_one")

In [43]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url
0,0,Q2.1,Neutral,bipoc,female,adult,anger,https://uwmadison.co1.qualtrics.com/ControlPan...
1,0,Q163.1,Happiness,white,male,child,happiness,https://uwmadison.co1.qualtrics.com/ControlPan...


In [44]:
df_emo_answers_long_m.to_csv('../clean_data/forced_choice_emotion_uw_students_long_format.csv', index=False)

# Sentiment score

In [45]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [46]:
sia = SentimentIntensityAnalyzer()

In [48]:
df_emo_answers_long_m['sentimentScore'] = df_emo_answers_long_m['emotion'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [49]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore
0,0,Q2.1,Neutral,bipoc,female,adult,anger,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0000
1,0,Q163.1,Happiness,white,male,child,happiness,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5574


# LMER

## Center predictors

In [50]:
df_emo_answers_long_m['sexC'] = df_emo_answers_long_m['sex'].replace({'female': -0.5, 'male': 0.5})
df_emo_answers_long_m['ageC'] = df_emo_answers_long_m['age'].replace({'child': -0.5, 'adult': 0.5})
df_emo_answers_long_m['ethnicityC'] = df_emo_answers_long_m['ethnicity'].replace({'bipoc': -0.5, 'white': 0.5})

In [51]:
df_emo_answers_long_m.head(2)

,participantId,photoId,emotion,ethnicity,sex,age,label,url,sentimentScore,sexC,ageC,ethnicityC
0,0,Q2.1,Neutral,bipoc,female,adult,anger,https://uwmadison.co1.qualtrics.com/ControlPan...,0.0000,-0.5,0.5,-0.5
1,0,Q163.1,Happiness,white,male,child,happiness,https://uwmadison.co1.qualtrics.com/ControlPan...,0.5574,0.5,-0.5,0.5


In [73]:
df_emo_answers_long_m_fil = df_emo_answers_long_m[df_emo_answers_long_m['label'] != 'attention']
df_emo_answers_long_m_fil = df_emo_answers_long_m_fil.reset_index(drop=True)

## Run model

In [78]:
# model = Lmer('sentimentScore ~ ethnicityC + sexC + (ethnicityC+sexC|participantId)', data=df_emo_answers_long_m)

In [ ]:
# model.fit()

In [76]:
# model_int = Lmer('sentimentScore ~ ethnicityC + sexC + (ethnicityC+sexC|participantId)', data=df_emo_answers_long_m)

In [77]:
# model_int.fit()

In [98]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [99]:
md = smf.mixedlm("sentimentScore ~  ethnicityC*sexC*ageC", df_emo_answers_long_m_fil, groups=df_emo_answers_long_m_fil["participantId"])

In [100]:
mdf = md.fit(method=["lbfgs"])

/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hes

In [101]:
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  sentimentScore
No. Observations:  9792     Method:              REML          
No. Groups:        51       Scale:               0.1682        
Min. group size:   192      Log-Likelihood:      inf           
Max. group size:   192      Converged:           Yes           
Mean group size:   192.0                                       
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             0.035                                    
ethnicityC            0.029    0.008  3.463 0.001  0.012  0.045
sexC                 -0.016    0.008 -1.932 0.053 -0.032  0.000
ethnicityC:sexC       0.041    0.017  2.465 0.014  0.008  0.073
ageC                 -0.030    0.008 -3.561 0.000 -0.046 -0.013
ethnicityC:ageC       0.040    0.017  2.416 0.016  0.

/home/pablo_ubuntu/miniconda3/envs/venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2705: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))


In [102]:
%load_ext rpy2.ipython

ModuleNotFoundError: No module named 'rpy2'